# Pakete importieren & Jupyter-Einstellungen

In [211]:
import pandas as pd
import numpy as np
import os

#Multiple Outputs für eine Codezelle
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"


# Working Directory einstellen

In [212]:
#Notebook liegt in dwh-etl/code - gehe in dwh-etl falls nötig
os.getcwd()[-12:]
if os.getcwd()[-12:] in ['dwh-etl/code', 'dwh-etl\code']:
    os.chdir("..")
os.getcwd()



'/home/florian/Daten/Dokumente/Studium/SS2021/Data Warehousing/Projekt/dwh-etl'

# Inputs einlesen

In [213]:
rki = pd.read_csv('input/rki/rki_2021_08_21_input.csv', sep=",")
lk = pd.read_csv('input/landkreise/Landkreise.csv', sep=",")
divi = pd.read_csv('input/divi/divi_2021_08_20_input.csv',sep=",")
impf = pd.read_csv('input/impfdashboard/impfungen_2021_08_20_input.tsv', sep="\t")
#Zeilen mit Nullwerten ("-") löschen
ag = pd.read_csv("input/destatis/altersgruppen.csv",sep=";", na_values='-').dropna()


# CSV-Ladedateien erstellen

## faelle_output.csv - COVID-19-Fallzahlen vom RKI

In [214]:
#Dimensionen einer Fallgruppe
dimensionen = ["IdLandkreis", "Altersgruppe", "Geschlecht", "Meldedatum"]

#Gruppieren & berechnen der Anzahl der gemeldeten Fälle pro Fallgruppe

#Neue Fälle
nf = rki.groupby(dimensionen, as_index=False).sum("AnzahlFall").sort_values(by=dimensionen)[dimensionen+["AnzahlFall"]]

#Neu Genesene
ng = rki.loc[rki.NeuGenesen != -9].groupby(dimensionen, as_index=False).sum("AnzahlGenesen").sort_values(by=dimensionen)[dimensionen+["AnzahlGenesen"]]

#Neue Todesfälle
nt = rki.loc[rki.NeuerTodesfall != -9].groupby(dimensionen, as_index=False).sum("AnzahlTodesfall").sort_values(by=dimensionen)[dimensionen+["AnzahlTodesfall"]]


#Spalten für Anzahl umbenennen
nf = nf.rename(columns={'AnzahlFall': 'Anzahl'})
ng = ng.rename(columns={'AnzahlGenesen': 'Anzahl'})
nt = nt.rename(columns={'AnzahlTodesfall': 'Anzahl'})


#Spalte "Falltyp" anhängen
nf = nf.assign(Falltyp_ID = 0)
ng = ng.assign(Falltyp_ID = 1)
nt = nt.assign(Falltyp_ID = 2)

#Zusammenfügen zu faelle
faelle = pd.concat([nf,ng,nt])

#Berliner Bezirke zu ganz Berlin aggregieren
berlin_agg = faelle[faelle.IdLandkreis // 1000 ==
                    11].groupby(["Altersgruppe", "Geschlecht", "Meldedatum", "Falltyp_ID"], as_index=False).sum("Anzahl").assign(IdLandkreis=11000)

#Daten der einzelnen Bezirke löschen
faelle.drop(faelle[(faelle.IdLandkreis // 1000 == 11) &
            (faelle.IdLandkreis % 11000 > 0)].index, inplace=True)

#aggregierte Daten anhängen
faelle = pd.concat([faelle, berlin_agg])


#Mappen von Spaltenwerten
faelle = faelle.replace({
        'Geschlecht': 
        {
            'M': 0,
            'W': 1,
            'D': 2,
            'unbekannt': 3
        },
        'Altersgruppe': 
        {
            'A00-A04': 0,
            'A05-A14': 1,
            'A15-A34': 2,
            'A35-A59': 3,
            'A60-A79': 4,
            'A80+': 5,
            'unbekannt': 6

        }
    }

)

#Umbennen von Spalten
faelle = faelle.rename(columns={'Geschlecht': 'Geschlecht_ID','Altersgruppe': 'AG_fein_ID'})



#Mappen von IdLandkreis auf Landkreis_ID aus der Basisdatenbank
faelle = pd.merge(faelle, lk, left_on= 'IdLandkreis', right_on = 'AGS', how='left')
faelle = faelle[["Landkreis_ID", "AG_fein_ID","Geschlecht_ID", "Meldedatum", "Falltyp_ID", "Anzahl"]]


#Ausgeben als CSV
faelle.to_csv('output/faelle_output.csv',index=True, index_label='Fall_ID', line_terminator='\r\n')


## betten_output.csv - Bettenbelegungen gemäß DIVI-Intensivregister

In [215]:
spalten = ["date", "gemeindeschluessel"]

#Bettenstatus frei
bf = divi[spalten + ["betten_frei"]].assign(Status_ID = 0).rename(columns={'betten_frei': 'Anzahl'})

#Bettenstatus belegt - Nicht COVID
bnc = divi[spalten]
bnc['Anzahl'] = divi['betten_belegt'] - divi['faelle_covid_aktuell']
bnc = bnc.assign(Status_ID = 1)

#Bettenstatus belegt - COVID invasiv
bci = divi[spalten + ["faelle_covid_aktuell_invasiv_beatmet"]
           ].assign(Status_ID = 2).rename(columns={'faelle_covid_aktuell_invasiv_beatmet': 'Anzahl'})

#Bettenstatus belegt - COVID nicht invasiv
bcni = divi[spalten]
bcni['Anzahl'] = divi['faelle_covid_aktuell'] - \
    divi['faelle_covid_aktuell_invasiv_beatmet']
bcni = bcni.assign(Status_ID = 3)


#Zusammenfuegen zu betten
betten = pd.concat([bf, bnc, bci, bcni])

#Spalten umbenennen
betten = betten.rename(columns={'date': 'Datum'})

#Mappen von gemeindeschluessel auf Landkreis_ID aus der Basisdatenbank
betten = pd.merge(betten, lk, left_on='gemeindeschluessel',
                  right_on='AGS', how='left')
betten.Landkreis_ID.dtype
lk.Landkreis_ID.dtype
betten = betten[["Datum", "Landkreis_ID", "Status_ID", "Anzahl"]]

#Ausgeben als CSV
betten.to_csv('output/betten_output.csv', index=True,
              index_label='Bett_ID', line_terminator='\r\n')


<ipython-input-215-cb9014b926d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bnc['Anzahl'] = divi['betten_belegt'] - divi['faelle_covid_aktuell']
<ipython-input-215-cb9014b926d8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bcni['Anzahl'] = divi['faelle_covid_aktuell'] - \


## impfungen_output.csv - Verabreichte Impfungen

In [216]:
#Tageswerte als Zeilendifferenzen bestimmen & mit Datum versehen
impfungen_output = pd.concat([impf.date, impf[['dosen_johnson_kumulativ',
                           'dosen_biontech_erst_kumulativ', 'dosen_biontech_zweit_kumulativ',
                            'dosen_moderna_erst_kumulativ', 'dosen_moderna_zweit_kumulativ',
                            'dosen_astrazeneca_erst_kumulativ', 'dosen_astrazeneca_zweit_kumulativ']].diff()], axis=1)

#NaN zu 0 wandeln
impfungen_output = impfungen_output.fillna(0)

#Spalten umbenennen - 'dosen_' & '_kumulativ' abschneiden & 'date' umbenennen
impfungen_output.columns = ['Datum'] + [x[6:-10] for x in impfungen_output.columns[1:]]

#Spalten für Werte zu Zeilen machen
impfungen_output = impfungen_output.melt(id_vars='Datum').rename(columns={'value': 'Anzahl'})

#Datentyp der Anzahl zu int ändern
impfungen_output["Anzahl"] = impfungen_output["Anzahl"].astype(int)

#Spalten anfügen
impfungen_output["Impfstoff_ID"] = impfungen_output.variable.map(
     {'biontech_erst': 0, 'biontech_zweit': 0, 'moderna_erst': 1, 'moderna_zweit': 1, 'astrazeneca_erst': 2, 'astrazeneca_zweit': 2, 'johnson': 3})

impfungen_output["Impfart_ID"] = impfungen_output.variable.map(
    {'biontech_erst': 0, 'biontech_zweit': 1, 'moderna_erst': 0, 'moderna_zweit': 1, 'astrazeneca_erst': 0, 'astrazeneca_zweit': 1, 'johnson': 1})

#Spalt
impfungen_output.drop(columns='variable', inplace=True)

#Ausgeben als CSV
impfungen_output.to_csv('output/impfungen_output.csv',
                        index=True, index_label='Impfungen_ID', line_terminator='\r\n')


## einwohner_output.csv - Daten zur Alterstruktur

In [218]:
#Datentyp auf int ändern - bei import wird automatisch float64 gewählt
for c in list(set(ag.columns) - {'Datum', 'ID', 'LK'}):
    ag[c] = ag[c].astype(int)

#Daten für Altersgruppen aggregieren

# --männlich

#0-14 Jahre
ag["männlich#0"] = ag["männlich unter 3 Jahre"] + ag["männlich 3 bis unter 6 Jahre"] + ag["männlich 6 bis unter 10 Jahre"] + ag["männlich 10 bis unter 15 Jahre"]
#15-34 Jahre
ag["männlich#1"] = ag["männlich 15 bis unter 18 Jahre"] + ag["männlich 18 bis unter 20 Jahre"] + ag["männlich 20 bis unter 25 Jahre"] + ag["männlich 25 bis unter 30 Jahre"] + ag["männlich 30 bis unter 35 Jahre"]
# 35-59 Jahre
ag["männlich#2"] = ag["männlich 35 bis unter 40 Jahre"] + ag["männlich 40 bis unter 45 Jahre"] + ag["männlich 45 bis unter 50 Jahre"] + ag["männlich 50 bis unter 55 Jahre"] + ag["männlich 55 bis unter 60 Jahre"]
# über 59
ag["männlich#3"] = ag["männlich 60 bis unter 65 Jahre"] + ag["männlich 65 bis unter 75 Jahre"] + ag["männlich 75 Jahre und mehr"]


# --weiblich

ag["weiblich#0"] = ag["weiblich unter 3 Jahre"] + ag["weiblich 3 bis unter 6 Jahre"] + ag["weiblich 6 bis unter 10 Jahre"] + ag["weiblich 10 bis unter 15 Jahre"]
#15-34 Jahre
ag["weiblich#1"] = ag["weiblich 15 bis unter 18 Jahre"] + ag["weiblich 18 bis unter 20 Jahre"] + ag["weiblich 20 bis unter 25 Jahre"] + \
    ag["weiblich 25 bis unter 30 Jahre"] + ag["weiblich 30 bis unter 35 Jahre"]
# 35-59 Jahre
ag["weiblich#2"] = ag["weiblich 35 bis unter 40 Jahre"] + ag["weiblich 40 bis unter 45 Jahre"] + ag["weiblich 45 bis unter 50 Jahre"] + \
    ag["weiblich 50 bis unter 55 Jahre"] + ag["weiblich 55 bis unter 60 Jahre"]
# über 59
ag["weiblich#3"] = ag["weiblich 60 bis unter 65 Jahre"] + ag["weiblich 65 bis unter 75 Jahre"] + ag["weiblich 75 Jahre und mehr"]

#Lösche nicht aggregierte Spalten
#ag.drop(ag.filter(regex='Jahre').columns, axis=1, inplace=True)

#Spalten mit Werten
cols = ag.filter(regex='#').columns


#Dataframe für Output
converted_ag = pd.DataFrame(
    columns=["Landkreis_ID", "AG_grob_ID", "Geschlecht_ID", "Einwohnerzahl"])


for row in ag.index:
    # mappe ag.LK auf Landkreise_ID aus lk
    lk_id = ""
    try:
        lk_id = lk[lk['AGS'] == ag["ID"][row]].iloc[0].Landkreis_ID
    except:
        print("no LK_ID found for {0}".format(ag["LK"][row]))

    # Berechne neue Zeilen aus den Wertspalten
    newRows = pd.concat([pd.DataFrame(data={
        "Einwohnerzahl": [int(ag[col][row])],
        "Geschlecht_ID": [col.find("männlich") != -1 and "0" or "1"],
        "AG_grob_ID": [col.replace("männlich#", "").replace("weiblich#", "")],
        "Landkreis_ID": [lk_id],
    }) for col in cols], ignore_index=True)
    converted_ag = converted_ag.append(newRows)

#Fortlaufenden Index generieren
converted_ag.reset_index(drop=True, inplace=True)


#Ausgeben als CSV
converted_ag.to_csv('output/einwohner_output.csv',
                      index=True, index_label='Einwohner_ID', line_terminator='\r\n')
